# Geocode & LGA Mapping - Australia

## Created by: Rebecca Jan
## Last updated: 2021-03-27

Set up notebook

In [22]:
import geopandas as gpd
from geopandas.tools import sjoin
from geopy import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import numpy as np
import pandas as pd
import shapely
from shapely.geometry import Point,  Polygon

Set path and load files

In [23]:
path='C:\\Users\\vicsxk6\\OneDrive - Department of Health and Human Services. Victoria\\Documents\\Python Scripts\\lga_mapping\\'
sitrep = pd.read_csv(path+"input\sitrep.csv", low_memory=False, sep =",")

AUS lookup sourced from - https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.003June%202020

In [24]:
aus_lga = gpd.read_file(path+"reference\\aus_lga_polygon_shp\LGA_2020_AUST.shp")

In [25]:
locator = Nominatim(user_agent='rebecca.jan@health.vic.gov.au')
sitrep['Address'] = sitrep['AddressLine1_yest'].map(str) + ', ' + sitrep['AddressLine2_yest'].map(str) + ', AUSTRALIA'

In [ ]:
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
sitrep['location'] = sitrep['Address'].apply(geocode)
sitrep['point'] = sitrep['location'].apply(lambda loc: tuple(loc.point) if loc else None)
sitrep[['latitude', 'longitude', 'altitude']] = pd.DataFrame(sitrep['point'].tolist(), index=sitrep.index)

Assign points to .csv data

In [ ]:
geometry = [Point(xy) for xy in zip(sitrep["longitude"], sitrep["latitude"])]
geo_df = gpd.GeoDataFrame(sitrep, crs="EPSG:4326", geometry=geometry)

Run inner join to pull in .shp data based on point

In [ ]:
aus_lga = aus_lga.to_crs(crs="EPSG:4326")
pointInPolys = sjoin(geo_df, aus_lga, how="left", op="within")

Sanity check data

In [ ]:
pointInPolys.info()
#pointInPolys.LGA_NAME20.unique()

Drop columns as needed

In [ ]:
pointInPolys.drop(pointInPolys.columns[[22,23,24,26,28]], axis = 1, inplace = True)

Convert columns to upper and export to csv

In [ ]:
pointInPolys.columns= pointInPolys.columns.str.upper()
pointInPolys.to_csv(path+"output\lga_mapped.csv", index=False)